In [ ]:
import csv

class Employee:
    id_counter = 0

    @classmethod
    def getEmployeeId(cls):
        return cls.id_counter

    def __init__(self,Idd=0, name='', age=0, position='', salary=0.0, email='example@example.com', gender='male'):
        Employee.id_counter += 1
        self.Id = Idd if Idd!=0 else Employee.id_counter
        self.setName(name)
        self.setAge(age)
        self.setPosition(position)
        self.setSalary(salary)
        self.setEmail(email)
        self.setGender(gender)

    def setName(self, name):
        self.__Name = name

    def setAge(self, age):
        self.__Age = age

    def setPosition(self, position):
        self.__position = position

    def setSalary(self, salary):
        self.__salary = salary

    def setEmail(self, email):
        while  '@'not in email:
            email = input("Invalid email,Enter Email again: ")
        self.__email=email

    def setGender(self, gender):
        self.__gender = gender

    def to_list(self):
        return [self.Id, self.__Name, self.__Age, self.__position, self.__salary, self.__email, self.__gender]

    def update(self, attr, value):
        if attr == "name":
            self.setName(value)
            return True
        elif attr == "age":
            self.setAge(int(value))
            return True
        elif attr == "position":
            self.setPosition(value)
            return True
        elif attr == "salary":
            self.setSalary(value)
            return True
        elif attr == "email":
            self.setEmail(value)
            return True
        elif attr == "gender":
            self.setGender(value)
            return True
        else:
            return False

    def __repr__(self):
        return f"ID: {self.Id}, Name: {self.__Name}, Age: {self.__Age}, Position: {self.__position}, Salary: {self.__salary}, Email: {self.__email}, Gender: {self.__gender}"

class FileHandler:
    
    def save_employee_details(self, filename,emp, mode='w'):
        try:
            with open(filename, mode,newline='') as file:# i must specify the newline
                writer = csv.writer(file)
                if file.tell() == 0:#it must be when the file is opened means inside the with statement
                    writer.writerow(["ID", "Name", "Age", "Position", "Salary", "Email", "Gender"])
                writer.writerow(emp.to_list())
        except FileNotFoundError:
                print(f"Error:file {filename} not exist.")
                    
    def read_from_file(self,filename):
        try:
            with open(filename, 'r') as file:
                emps= file.read().splitlines()
            return emps
        except FileNotFoundError:
                print(f"Error:file {filename} not exist.")
                  
    def update_employee_file(self,filename,lines):
        try:
            with open(filename,'w',newline='')as file:
#                 file.write("\n")
#                 for line in lines:
#                     file.write(line+"\n")
                 writer = csv.writer(file)
                 writer.writerows(lines)
        except Exception as e:
            print(f" Error : {e}")
                
    def convert_file_to_object(self,lines):
        try:
            emp= Employee(int(lines[0]),lines[1],int(lines[2]),lines[3],float(lines[4]),lines[5],lines[6])
            return emp
        except IndexError as e:
             raise ValueError(f"Error in cell : {lines}") from e 
                
    def read_last_line(self,filename):
        try:
            with open(filename, 'r') as file:
                lines = file.readlines()  
                return lines[-1] if lines else None  
        except FileNotFoundError:
             print(f" file {filename} not exist")
        return 


class ManagementSystem: 
    def __init__(self):
        self.fileHandler=FileHandler()
            
    def add_employee(self):
        try:
            name = input("Enter Name: ")
            age = int(input("Enter Age: "))
            position = input("Enter Position: ")
            salary = float(input("Enter Salary: "))
            email = input("Enter Email: ")
            gender = input("Enter Gender: ")
            emp = Employee(0,name, age, position, salary, email, gender)
            last_line=self.fileHandler.read_last_line("EmployeesFile.csv")
            if las_line!=None:
                emp.Id=int(last_line[0])+1#so as not have repeated id when close the program and run it again
            else:
                emp.Id=1
            self.fileHandler.save_employee_details("EmployeesFile.csv", emp,"a")
            print(f"\nEmployee {name} added successfully.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def update_employee(self):
        try:
            emp_id = int(input("Enter the id for the employee you want to update: "))
            emps=self.fileHandler.read_from_file('EmployeesFile.csv')
            flag=False
            for i, emp in enumerate(emps):
                if i == 0:  # Skip the first row (header)
                    continue
                emp=emp.split(',')
                if int(emp[0]) == emp_id:                   
                    emp = self.fileHandler.convert_file_to_object(emp)
                    attr = input("Enter attribute to update(name-age-position-salary-email-gender): ")
                    value = input("Enter new value: ")
                    flag=emp.update(attr, value)
                    if flag==True:
                        print(emp)
                        #self.fileHandler.save_employee_details( 'EmployeesFile.csv',emp, 'w')
                        emps[i] = ','.join(map(str, emp.to_list()))#update the list containing the employee
                        self.fileHandler.update_employee_file('EmployeesFile.csv',[row.split(',') for row in emps])#emps)
                        print(f"\nEmployee updated successfully.")
                    else:
                        print("Attribute is invalid")
                    return
            if flag==True:    
                 print("Employee not found")
        except Exception as e:
            print(f"Error : {e}")
    
    def delete_employee(self):
        try:
            emp_id = int(input("Enter the id for the employee you want to delete: "))
            emps=self.fileHandler.read_from_file('EmployeesFile.csv')
            found=False
            for i, emp in enumerate( emps):
                if i==0 :
                    continue
                emp=emp.split(',')
                if emp[0]==str(emp_id):
#                     emp = ','.join(map(str, emp.to_list()))
#                     emps.remove(emp)
#                     del emp
                      found=True
                    continue
                    #self.fileHandler.update_employee_file("EmployeesFile.csv",emps)
                    #print("Employee deleted successfully")
                    #return
                else:
                    emp = self.fileHandler.convert_file_to_object(emp)
                    self.fileHandler.save_employee_details("EmployeesFile.csv", emp)
            #print("Failed to delete Employee,Employee not found ")
            if found==True:
                print("Employee deleted successfully")
            else:
                print("Failed to delete Employee, Employee not found")
        except Exception as e:
            print(f"Error : {e}")
    
    def search_employee(self):
        try:
                emp_id = int(input("Enter the id for the employee you want to search for: "))
                emps=self.fileHandler.read_from_file('EmployeesFile.csv')
                for emp in emps:
                    emp.split(',')
                    if emp[0]==str(emp_id):
                        print(emp)
                        return
    
                print('Employee not found')
        except Exception as e:
            print(f"Error : {e}")
            
    def load_all_employees(self,filename):
        try:
            emps= self.fileHandler.read_from_file(filename)
            for emp in emps:
                row = emp.split(',')
                print(row) 
        except FileNotFoundError:
            print(f" file {filename} not exist.")
        except Exception as e:
            print(f"Error : {e}")
        return 

    
def display_menu():
    print("\nMenu:")
    print("1. Add Employee")
    print("2. Update Employee")
    print("3. Delete Employee")
    print("4. Search for an Employee")
    print("5. List all Employees")
    print("6. Exit")

def main():
    while True:
        display_menu()
        choice = input("Enter your choice: ")
        if choice == "1":
            ms.add_employee()
        elif choice == "2":
            ms.update_employee()
        elif choice == "3":
            ms.delete_employee()
        elif choice == "4":
            ms.search_employee()
        elif choice == "5":
            ms.load_all_employees('EmployeesFile.csv')
        elif choice == "6":
            break
        else:
            print("Invalid choice. Please try again.")
            
# Main menu
ms = ManagementSystem()
ms.load_all_employees('EmployeesFile.csv')

main() 
# e=Employee(0,'ata',29,'sgh',3000,'aya@com','femail')
#  print(e)

['ID', 'Name', 'Age', 'Position', 'Salary', 'Email', 'Gender']
['3', 'fd', '34', 'f', '55.0', 'f@', 'dfd']

Menu:
1. Add Employee
2. Update Employee
3. Delete Employee
4. Search for an Employee
5. List all Employees
6. Exit
Enter your choice: 1
Enter Name: aya
Enter Age: 3
Enter Position: r
Enter Salary: 4
Enter Email: @
Enter Gender: er

Employee aya added successfully.

Menu:
1. Add Employee
2. Update Employee
3. Delete Employee
4. Search for an Employee
5. List all Employees
6. Exit
Enter your choice: 5
['ID', 'Name', 'Age', 'Position', 'Salary', 'Email', 'Gender']
['3', 'fd', '34', 'f', '55.0', 'f@', 'dfd']
['4', 'aya', '3', 'r', '4.0', '@', 'er']

Menu:
1. Add Employee
2. Update Employee
3. Delete Employee
4. Search for an Employee
5. List all Employees
6. Exit
